In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

import flow

In [2]:

df = flow.prep_county_ethanol_production_data()
df

,FIPS,State,County,biomass_production_bbtu
0,01001,AL,Autauga County,0.0
1,01003,AL,Baldwin County,0.0
2,01005,AL,Barbour County,0.0
3,01007,AL,Bibb County,0.0
4,01009,AL,Blount County,0.0
...,...,...,...,...
3218,72151,PR,Yabucoa Municipio,0.0
3219,72153,PR,Yauco Municipio,0.0
3220,78010,VI,St. Croix County,0.0
3221,78020,VI,St. John County,0.0


In [5]:
df = flow.prep_state_fuel_production_data()
df.sum()

State                   AKALARAZCACOCTDCDEFLGAHIIAIDILINKSKYLAMAMDMEMI...
coal_production                                                17931021.0
biomass_production                                              1998016.0
natgas_production                                              31276976.0
petroleum_production                                           16469427.0
dtype: object

In [ ]:
df = flow.prep_fuel_demand_data()

for col in df.columns:
    print(col)

In [ ]:
                    "NUETB": "nuclear_production",  #Nuclear energy consumed for electricity generation, total (BBTU)
                    "GETCB": "geothermal_production",  #Geothermal energy total consumption (BBTU)
                    "HYTCB": "hydropower_production",  #Hydropower total consumption (BBTU)
                    "WYTCB": "wind_production",  #Wind energy total consumption (BBTU)
                    "SOTCB": "solar_production"  #Solar energy total consumption (BBTU)

In [4]:
is_NaN = df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df[row_has_NaN]
rows_with_NaN

,FIPS,Refuse,Surface,Underground


In [ ]:
df = flow.prep_electricity_generation()
df.isnull().values.sum()
df2 = flow.prep_county_identifier()


In [ ]:
df2.to_csv(r'C:\Users\mong275\Local Files\Sankey_Notebook_Development\SankeyFiles\run2.csv')

In [ ]:
is_NaN = df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df[row_has_NaN]
rows_with_NaN

In [ ]:
city_list = ['VAchesapeakecity','VAportsmouthcity','VAhopewellcity','VAalexandriacity',
           'VAcovingtoncity','VAsuffolkcity','VAharrisonburgcity','VAsalemcity',
           'VAlynchburgcity','VAdanvillecity','VAmanassascity','VAhamptoncity',
           'VAvirginiabeachcity','VAbristolcity', 'MOstlouiscity']

for i in city_list:
    df_plant["identifier"] = np.where(df_plant["identifier"] == i, df_plant["identifier"].str.replace('city', '', regex=True), df_plant["identifier"])



<h1> Public Water Supply </h1>

## Water in Public Water Supply

### Public Water Supply Withdrawals

The USGS 2015 dataset provides public water supply withdrawals directly under data codes 'PS-WGWFr', 'PS-WSWFr','PS-WGWSa', 'PS-WSWSa' for each county.

In [ ]:
df = flow.prep_water_use_2015(variables=['State','County','PS-WGWFr', 'PS-WSWFr','PS-WGWSa', 'PS-WSWSa'])
df.head()               

### Public Water Supply Deliveries, Exports, and Imports

The USGS 2015 dataset provide public water supply data in three categories: (1) total public water supply withdrawals, (2) public water supply deliveries to domestic end users, and (3) public water supply deliveries to thermoelectric cooling.

The 1995 USGS dataset provides public water supply deliveries to domestic, industrial, commercial, thermoelectric cooling, and losses/exports separately. To estimate the deliveries to commercial and industrial customers in 2015, the ratio of each to the sum of deliveries to residential and thermoelectric cooling is calculated from the 1995 data. These ratios are applied to the 2015 residential and thermoelectric cooling public water supply delivery values. 

Additionally, public water suppy exports and imports are calculated. When the sum of water deliveries is greater than the water withdrawn for public water supply, the difference is considered to be imported. When water withdrawn is greater, excess water after deliveries is considered to be exported.

Caveats and assumptions:
* This methodology assumes that if no public water supply was delivered to residential or thermoelectric cooling in 1995, then deliveries to industrial and commercial sectors would likewise be zero.

Potential mods
 for import pws counties, take 15% of withdrawals and imports as losses and add it on to the top of imports
 for export pws counties, take 15% of withdrawals as losses, take it out of exports, any remaining put as imports

In [ ]:
df = flow.calc_pws_discharge()
df.head()

## Energy in Public Water Supply

# Wastewater Treatment
Wastewater treatment facility data is provided by EPA for the year 2012. These values are assumed for 2015. Values are provided on a treatment plant level with data on the following variables: 
Flow:
    * Total water flows (MGD) treated by the facility
    * Total infiltration flows treated by the facility (leaks into pipes and conduits)
Treatment:
    * Treatment type of each treatment plant (primary, secondary, advanced)
Discharge:
    * discharge location of each plant, broken down by percent of flow to each discharge location (e.g., ocean discharge, suface discharge, irrigation reuse)
Location:
    * county information is included for each power plant
    
This data allows us to determine total water flows processed and discharged by wastewater facilities at a county level by treatment type. The treatment type allows us to calculate the electricity use by county for wastewater treatment,

## Water in Wastewater Treatment

### Water Inflows to Wastewater Treatment Facilities
Total flows processed by wastewater treatment facilities are provided in the EPA dataset directly. However, this data will have discrepencies from the water data delivered to various sectors. To estimate water flows from residential, commercial, and industrial sectors to wastewater treatment facilities, an assumption is made that all water delivered from public water supply to these sectors is sent to wastewater treatment facilities.

### Quantity of Wastewater Processed by Treatment Type
Wastewater treatment plant treatment type is provided in the EPA data directly. These values are binned for simplification into no treatment, primary, secondary, and advanced treatment. When no treatment type was provided for a facility, it was assumed that they were secondary treatment facilities.

To see the amount of water (MGD) processed within each county by treatment type, run the cell below.

In [ ]:
df = flow.prep_wastewater_data()
df = df[['FIPS','no_treatment','ww_adv','ww_prim','ww_sec']]
df.head()

### Wastewater Treatment Facility Discharge

Wastewater treatment plant discharge location is provided in the EPA data directly. These values are binned for simplification into the following categories:

* 'outfall to surface waters': 'ww_sd',
* 'ocean discharge': 'ww_od',
* 'deep well': 'ww_gd',
* "reuse: industrial": 'ww_in',
* 'evaporation': 'ww_evap',
* 'spray irrigation': 'ww_ir',
* 'overland flow no discharge': 'ww_ww',
* 'overland flow with discharge': 'ww_sd',
* 'discharge to another facility': 'ww_ww',
* 'combined sewer overflow (cso) discharge': 'ww_sd',
* 'other': 'ww_sd',
* 'discharge to groundwater': 'ww_gd',
* 'no discharge, unknown': 'ww_ww',
* 'reuse: irrigation': 'ww_ir',
* 'reuse: other non-potable': 'ww_sd',
* 'reuse: indirect potable': 'ww_sd',
* 'reuse: potable': 'ww_ps',
* 'reuse: groundwater recharge': 'ww_gd'

In [ ]:
df = flow.prep_wastewater_data()
df = df[['FIPS','ww_evap','ww_gd','ww_in','ww_ir','ww_od','ww_ps','ww_sd','ww_ww']]
df.head()

## Energy in Wastewater Treatment

Wastewater treatment facilities have various levels of treatment from primary to advanced. Each treatment level has an increasing level of energy intensity to process the water. Intensity value estimates for wastwater treatment were used from Pabi et al. [17].

# Agriculture

## Water in Agriculture

## Energy in Agriculture

Energy use in agriculture consists of groundwater pumping, surface water pumping, and interbasin transers. USDA's Farm and Ranch Irrigation Survey (FRIS) provides state-by-state data on irrigation groundwater depth and average irrigation pressurization levels for irrigation within a state, enabling the calculation of pump electricity consumption for both groundwater and surface water pumping [13]. The 2013 survey is the closest year available to 2015 values. It is assumed that values do not vary significantly between the two years.

The methodology for calculating groundwater and surface water pumping energy is described in Tidwell et al [14]. However, where that publication uses well depth to water to calculate total differential height, the total well depth is used here as a way to offset some of the losses due to friction that would occur in the piping, as described in Lawrence Berkeley National Laboratory (LBNL) Home Energy Saver & Score: Engineering Documentation [15].

Pump efficiency is assumed to be the average (46.5%) of the range (34-59%) listed in [14]. State-level intensity rates are calculated here and applied to the county level water in agriculture values. The general methodology implemented here to determine total energy in agriculture is adapted from [5], not including the additions previously mentioned.

### Groundwater & Surface Water Pumping

### Interbasin Transfers

# Electricity Generation

## Energy in Electricity Generation

US Energy Information Administration (EIA) 923 data for 2015 is used to determine electricity generation by generator type within each interconnect [12]. The datafile contains information on generation at the plant-level. EIA additionally provides data on which North American Electric Reliability Corporation (NERC) balancing region each plant belongs to. By mapping each NERC region to its associated interconnect, generation within each interconnect can be determined directly.

From EIA, the AER Fuel Type Code is used to determine generator type. EIA provides generator type information at a high level of detail. For this reason different generator types are grouped for simplicity using a generator key (GenType923Key.xlsx). For example, different types of natural gas generators are all grouped into "Natural Gas" as a type.

Energy production is provided as "Net Generation (Megawatthours)" in the EIA dataset, "Total Fuel Consumption (MMBTU)" is the MMBTU equivalent of fuel going to each plant for electricity generation. The difference between these two values indicates the amount of energy generation that does not go to energy losses.

## will be filled in with calculate function

## Water in Electricity Generation

# End Use Sectors

## Residential Sector

### Water in Residential Sector

### Energy in Residential Sector

#### Energy (electricity) Demand in Residential Sector

##### weighted by population. <b> consider splitting out by usage if data is available

In [ ]:
df = flow.prep_electricity_demand_data()
df

#### Energy (fuel) Demand in Residential Sector

## Commercial Sector

### Water in Commercial Sector

### Energy in Commercial Sector

## Industrial Sector

### Water in Industrial Sector

### Energy in Industrial Sector

## Transportation Sector

### Energy in Energy Sector

# crude oil production by county fillers
Idaho: https://ogcc.idaho.gov/wp-content/uploads/sites/3/2017/06/2016-10-6-state-of-idaho-releases-oil-gas-production-data.pdf
100% of Idaho petroleum and natural gas is from Payette County

Alaska,
https://www.commerce.alaska.gov/web/aogcc/Data.aspx
almost all oil is produced along the north slope. Calculated from oil production download. The rest is on the kenai peninsula (cook inlet basin)

Maryland
https://www.eia.gov/state/analysis.php?sid=MD#34
assumed 50/50 split between Garret and Allegany Counties

Nevada
https://gisweb.unr.edu/OilGas/
Assumed to be in nye county with producing oil wells (only 4 btu)

Oregon
https://www.oregongeology.org/mlrr/oilgas-report.htm
Only one county (columbia county)

# Intensities & Percentages

#### Conveyance Losses as a Percent of Water Withdrawn for Irrigation
Conveyance loss fraction for irrigation is calculated from the USGS 1995 water use dataset. It is calculated as the total conveyance losses (MGD) divided by the total water withdrawn (MGD) for irrigation. Looking at the fractions at the county level shows that a small subset of counties have unreliable values where conveyance losses are far greater than the amount withdrawn for irrigation. As a result, the ability to cap the loss fraction at a specified amount it provided and replace it with the average of the specified region. The default loss cap fraction is .90.

In [ ]:
df = flow.prep_water_use_1995()
df = flow.calc_conveyance_loss_frac(df)
df.head()

##### Change in loss cap

In [ ]:
df = flow.prep_water_use_1995()
df = flow.calc_conveyance_loss_frac(df, loss_cap_amt=.20)
df.head()

In [ ]:
help(flow.plot_bar)

In [ ]:
flow.plot_bar(df, "State","IR_CLoss_Frac", "State", y_axis_title = "Irrigation Conveyance Loss Fraction", x_axis_title="State")

#### Water Use Intensity for Hydroelectric Generation

In [ ]:
df = flow.calc_hydroelectric_water_intensity(intensity_cap=True, intensity_cap_amt=165, region_avg=False)

df.head()

In [ ]:
flow.plot_bar(df, "State","HY_IF", "State")